In [3]:
import pandas as pd
import numpy as np

# Fonction pour optimiser les types de données
def optimize_dtypes(df):
    for col in df.columns:
        if df[col].dtype == 'int64':
            df[col] = df[col].astype('int32')
        elif df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
    return df

# Charger et optimiser les datasets plus petits
movies_df = optimize_dtypes(pd.read_csv('/Film_recom/movies.csv'))
links_df = optimize_dtypes(pd.read_csv('/Film_recom/links.csv'))
genome_tags_df = optimize_dtypes(pd.read_csv('/Film_recom/genome-tags.csv'))
tags_df = optimize_dtypes(pd.read_csv('/Film_recom/tags.csv'))

# Fusionner movies et links
movies_links_df = pd.merge(movies_df, links_df, on='movieId', how='left')

def process_chunk(chunk):
    return optimize_dtypes(chunk)

# Traiter les ratings
ratings_df = pd.DataFrame()
for chunk in pd.read_csv('/Film_recom/ratings.csv', chunksize=1000000):
    ratings_df = pd.concat([ratings_df, process_chunk(chunk)], ignore_index=True)

# Fusionner movies_links avec ratings
final_df = pd.merge(movies_links_df, ratings_df, on='movieId', how='right')

# Fusionner avec tags
final_df = pd.merge(final_df, tags_df, on=['movieId', 'userId'], how='left')

# Traiter les genome_scores
genome_scores_df = pd.DataFrame()
for chunk in pd.read_csv('/Film_recom/genome-scores.csv', chunksize=1000000):
    genome_scores_df = pd.concat([genome_scores_df, process_chunk(chunk)], ignore_index=True)

# Fusionner genome_scores avec genome_tags
genome_df = pd.merge(genome_scores_df, genome_tags_df, on='tagId', how='left')

# Agréger les données de genome pour chaque film
genome_agg = genome_df.groupby('movieId').agg({
    'relevance': ['mean', 'max'],
    'tag': lambda x: ', '.join(x.dropna().unique())
}).reset_index()

genome_agg.columns = ['movieId', 'avg_relevance', 'max_relevance', 'genome_tags']

# Fusionner l'agrégation genome avec le DataFrame final
final_df = pd.merge(final_df, genome_agg, on='movieId', how='left')

# Définir l'ordre des colonnes souhaité
desired_columns = ['movieId', 'title', 'genres', 'imdbId', 'tmdbId', 'userId', 'rating', 'timestamp', 'tag', 'avg_relevance', 'max_relevance', 'genome_tags']

# Vérifier quelles colonnes sont disponibles dans le DataFrame final
available_columns = set(final_df.columns)

# Filtrer pour ne garder que les colonnes qui existent réellement
columns_order = [col for col in desired_columns if col in available_columns]

# Réorganiser les colonnes
final_df = final_df[columns_order]

# Afficher les colonnes manquantes
missing_columns = set(desired_columns) - set(columns_order)
if missing_columns:
    print(f"Colonnes manquantes : {missing_columns}")

# Supprimer les doublons
final_df = final_df.drop_duplicates()

# Réinitialiser l'index
final_df = final_df.reset_index(drop=True)

# Vérifier le résultat
print(final_df.info())
print(final_df.head())

# Sauvegarder le dataset fusionné
final_df.to_csv('/Film_recom/merged_movie_data.csv', index=False)

Colonnes manquantes : {'timestamp'}
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20265625 entries, 0 to 20265624
Data columns (total 11 columns):
 #   Column         Dtype  
---  ------         -----  
 0   movieId        int32  
 1   title          object 
 2   genres         object 
 3   imdbId         int32  
 4   tmdbId         float32
 5   userId         int32  
 6   rating         float32
 7   tag            object 
 8   avg_relevance  float32
 9   max_relevance  float32
 10  genome_tags    object 
dtypes: float32(4), int32(3), object(4)
memory usage: 1.1+ GB
None
   movieId                                              title  \
0        2                                     Jumanji (1995)   
1       29  City of Lost Children, The (Cité des enfants p...   
2       32          Twelve Monkeys (a.k.a. 12 Monkeys) (1995)   
3       47                        Seven (a.k.a. Se7en) (1995)   
4       50                         Usual Suspects, The (1995)   

                           

In [1]:
# Cellule 1: Importation
from merged_movie_data_pipeline import MergedMovieDataPipeline

# Cellule 2: Exécution du pipeline
pipeline = MergedMovieDataPipeline('F:/Film_recom/merged_movie_data.csv')
pipeline.run_pipeline()
processed_data = pipeline.get_data()

# Cellule 3: Description des données (optionnel)
pipeline.describe_data()

# Cellule 4: Affichage des premières lignes des données traitées
print(processed_data.head())

# Cellule 5: Affichage des informations sur le DataFrame
print(processed_data.info())

NameError: name 'MergedMovieDataPipeline' is not defined